In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\lp001.xml,453,707,332,392
1,./images\lp002.xml,485,722,449,574
2,./images\lp003.xml,38,136,69,121
3,./images\lp004.xml,98,190,105,130
4,./images\lp005.xml,105,200,103,130


In [3]:
import xml.etree.ElementTree as xet

In [4]:
filename = df['filepath'][0]
filename

'./images\\lp001.xml'

In [5]:
def getFileName(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('./images', filename_image)
    return filepath_image

In [6]:
getFileName(filename)

'./images\\lp001.jpg'

In [7]:
image_path = list(df['filepath'].apply(getFileName))

In [8]:
img = cv2.imread(image_path[0])
#cv2.namedWindow('example', cv2.WINDOW_NORMAL)
#cv2.imshow('example', img)
#cv2.waitKey()
#cv2.destroyAllWindows()

In [9]:
cv2.rectangle(img, (453, 332), (707, 392), (0, 255, 0), 3)
cv2.namedWindow('example')
cv2.imshow('example', img)
cv2.waitKey()
cv2.destroyAllWindows()

### Data Preprocessing

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [12]:
labels = df.iloc[:, 1:].values
labels

array([[ 453,  707,  332,  392],
       [ 485,  722,  449,  574],
       [  38,  136,   69,  121],
       ...,
       [ 342,  604,  948, 1030],
       [ 342,  426,  194,  227],
       [ 149,  345,  251,  303]], dtype=int64)

In [17]:
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[0]
    img_arr = cv2.imread(image)
    h, w, d = img_arr.shape
    # preprocessing
    load_image = load_img(image, target_size = (224, 224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr / 255.0 #normalization
    # normalization to labels
    xmin, xmax, ymin, ymax = labels[ind]
    nxmin, nxmax = xmin / w, xmax / w
    nymin, nymax = ymin / h, ymax / h
    label_norm = (nxmin, nxmax, nymin, nymax) # normalized output
    
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [19]:
X = np.array(data, dtype = np.float32)
y = np.array(output, dtype = np.float32)

In [20]:
X.shape, y.shape

((275, 224, 224, 3), (275, 4))

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, random_state = 0)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((220, 224, 224, 3), (55, 224, 224, 3), (220, 4), (55, 4))

### Deep Learning Model

In [27]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [28]:
inception_resnet = InceptionResNetV2(weights = "imagenet", 
                                     include_top = False, 
                                     input_tensor = Input(shape = (224, 224, 3)))
inception_resnet.trainable = False
# ---------------------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation = 'relu')(headmodel)
headmodel = Dense(250, activation = 'relu')(headmodel)
headmodel = Dense(4, activation = 'sigmoid')(headmodel)
# model
model = Model(inputs = inception_resnet.input, outputs = headmodel)

219070464/219055592 [==============================] - 8s 0us/step


In [30]:
# compile model
model.compile(loss = 'mse', 
              optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-4))


In [31]:
from tensorflow.keras.callbacks import TensorBoard

In [32]:
tfb = TensorBoard('object_detection')

In [35]:
history = model.fit(x = x_train, 
                    y = y_train, 
                    batch_size = 10, 
                    epochs = 100, 
                    validation_data = (x_test, y_test),
                    callbacks = [tfb])

Epoch 1/100
22/22 [==============================] - 41s 2s/step - loss: 0.3322 - val_loss: 0.3045
Epoch 2/100
22/22 [==============================] - 34s 2s/step - loss: 0.3322 - val_loss: 0.3045
Epoch 3/100
22/22 [==============================] - 35s 2s/step - loss: 0.3322 - val_loss: 0.3045
Epoch 4/100
22/22 [==============================] - 39s 2s/step - loss: 0.3322 - val_loss: 0.3045
Epoch 5/100
22/22 [==============================] - 36s 2s/step - loss: 0.3322 - val_loss: 0.3045
Epoch 6/100
22/22 [==============================] - ETA: 0s - loss: 0.3322

KeyboardInterrupt: 